In [1]:
import pandas as pd
import numpy as np
import ensemble_boxes 
from pycocotools.coco import COCO
import itertools
import os
from tqdm import tqdm

In [2]:
num_file = 5
algorithms = ['nms', 'soft_nms', 'non_maximum_weighted', 'weighted_boxes_fusion']
iou_thresholds = np.arange(0.1, 0.91, 0.1).tolist()
output_name_format = '1011_deta_{num_file}_fold_{algorithm}_{iou_threshold}.csv'
output_fold = 'result'

submission_file_format = '../../result/jozhang97/deta-swin-large_{i}/deta-swin-large_{i}.csv'
annotation_path = '/data/ephemeral/home/data/dataset/test.json'

submission_file_paths = []

for i in range(num_file):
    submission_file_path = submission_file_format.format(i = i)
    submission_file_paths.append(submission_file_path)

submission_df = [pd.read_csv(file) for file in submission_file_paths]
image_ids = submission_df[0]['image_id'].tolist()

coco = COCO(annotation_path)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [3]:
def get_box_score_label(submission_df, image_id, image_info):
    boxes_list = []
    scores_list = []
    labels_list = []

    # 각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()

        if len(predict_list)==0 or len(predict_list)==1:
            continue

        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []

        for box in predict_list[:, 2:6].tolist():
            # box의 각 좌표를 float형으로 변환한 후 image의 넓이와 높이로 각각 정규화
            image_width = image_info['width']
            image_height = image_info['height']

            box[0] = float(box[0]) / image_width
            box[1] = float(box[1]) / image_height
            box[2] = float(box[2]) / image_width
            box[3] = float(box[3]) / image_height

            box = np.clip(box, 0, 1).tolist()
            box_list.append(box)

        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))

    return boxes_list, scores_list, labels_list


def get_prediction(boxes, scores, labels, image_width, image_height):
    prediction_string = ""
    for box, score, label in zip(boxes, scores, labels):
        xmin, ymin = box[0] * image_width, box[1] * image_height
        xmax, ymax = box[2] * image_width, box[3] * image_height
        prediction_string += f'{label:.0f} {score:.4f} {xmin:.1f} {ymin:.1f} {xmax:.1f} {ymax:.1f}'
    return prediction_string

In [5]:
conf = list(enumerate(list(itertools.product(algorithms, iou_thresholds)))) # ensemble할 조건 리스트
results = [{'prediction_strings': [], 'file_names': []} for i in range(len(conf))] # 결과 저장 리스트 선언

# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.4

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(tqdm(image_ids)):    
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []

    image_info = coco.loadImgs(i)[0]
    image_width = image_info['width']
    image_height = image_info['height']

    boxes_list, scores_list, labels_list = get_box_score_label(submission_df, image_id, image_info)
    
    # 예측 box가 있다면 ensemble 수행
    if len(boxes_list):
        # ensemble에 필요한 인자: [box의 lists, confidence score의 lists, label의 list, iou에 사용할 threshold]
        for j, (algorithm, iou_threshold) in conf:
            boxes, scores, labels = getattr(ensemble_boxes, algorithm)(boxes_list, scores_list, labels_list, iou_thr=iou_threshold)
            prediction_string = get_prediction(boxes, scores, labels, image_width, image_height)

            results[j]['prediction_strings'].append(prediction_string)
            results[j]['file_names'].append(image_id)    

  0%|          | 5/4871 [00:42<5:48:56,  4.30s/it] 

In [5]:
prediction_string

'7 0.8268 699.8 141.3 890.2 334.91 0.7998 286.4 289.9 537.6 514.91 0.7986 201.0 412.7 611.4 732.31 0.7871 240.9 598.0 522.5 814.81 0.7237 185.3 305.2 289.5 473.03 0.7224 843.0 396.8 956.5 651.41 0.7213 190.2 294.9 610.5 738.11 0.6488 188.2 288.6 512.5 473.67 0.6017 283.1 674.4 423.1 725.35 0.5948 283.4 674.3 423.9 725.51 0.5939 326.7 281.2 441.6 321.01 0.5741 233.5 288.7 504.4 379.41 0.5587 522.7 422.5 593.1 507.81 0.5355 233.5 289.8 505.4 411.71 0.5257 496.9 308.7 594.3 447.21 0.4869 519.6 425.6 612.4 586.61 0.4778 233.5 287.8 505.0 395.31 0.4644 496.9 307.1 594.6 465.20 0.4574 266.9 425.5 471.0 521.21 0.4334 234.3 291.5 306.6 382.41 0.4321 523.6 393.0 552.1 462.71 0.4137 329.0 280.3 440.5 310.71 0.4103 277.3 298.0 604.8 586.31 0.3923 525.7 394.1 550.8 459.81 0.3781 306.1 287.2 443.4 327.11 0.3739 522.7 393.6 550.9 473.71 0.3667 439.7 294.0 503.5 341.31 0.3614 314.1 282.1 504.0 341.40 0.3549 282.1 427.4 470.6 521.80 0.3464 185.2 305.0 290.0 472.71 0.3225 521.5 467.9 611.8 582.51 0.313

In [10]:
# 결과 저장하기
os.makedirs(output_fold, exist_ok=True)

for i, (algorithm, iou_threshold) in conf:
    output_name = output_name_format.format(num_file=num_file, algorithm=algorithm, iou_threshold=iou_threshold)
    output_path = os.path.join(output_fold, output_name)

    submission = pd.DataFrame()
    submission['PredictionString'] = results[i]['prediction_strings']
    submission['image_id'] = results[i]['file_names']

    submission.to_csv(output_path, index=False)
    submission.head()